# Two Player Quantum Lower Bound

### Description:
Compute a lower bound on the quantum value of a nonlocal game.

Calculates a lower bound on the maximum value that the specified nonlocal
game can take on in quantum mechanical settings where Alice and Bob each
have access to a d-dimensional quantum system.

This approach works by starting with a randomly-generated POVM for Bob, and
then optimizing Alice's POVM and the shared entangled state. Then Alice's
POVM and the entangled state are fixed, and Bob's POVM is optimized. And so
on, back and forth between Alice and Bob until convergence is reached.

Note that the algorithm is not guaranteed to obtain the optimal local bound
and can get stuck in local minimum values. The alleviate this, it is run
more than once to run the algorithm some pre-specified number of
times and keep the highest value obtained.

### Explanation:
The algorithm is based on the alternating projections algorithm as it can
be applied to Bell inequalities as shown in [1].

The alternating projection algorithm has also been referred to as the 
"see-saw" algorithm as it goes back and forth between the following two 
semidefinite programs:

$$
\textbf{SDP-1:} \qquad
\begin{align*}
                \text{maximize:} \quad & \sum_{(x,y \in \Sigma)} \pi(x,y)
                                         \sum_{(a,b) \in \Gamma}
                                         V(a,b|x,y)
                                         \langle B_b^y, A_a^x \rangle \\
                \text{subject to:} \quad & \sum_{a \in \Gamma_{\mathsf{A}}} =
                                           \tau, \qquad \qquad
                                           \forall x \in \Sigma_{\mathsf{A}}, \\
                                   \quad & A_a^x \in \text{Pos}(\mathcal{A}),
                                           \qquad
                                           \forall x \in \Sigma_{\mathsf{A}}, \
                                           \forall a \in \Gamma_{\mathsf{A}}, \\
                                           & \tau \in \text{D}(\mathcal{A}).
\end{align*}
$$

$$
\textbf{SDP-2:} \qquad
\begin{align*}
                \text{maximize:} \quad & \sum_{(x,y \in \Sigma)} \pi(x,y)
                                         \sum_{(a,b) \in \Gamma} V(a,b|x,y)
                                         \langle B_b^y, A_a^x \rangle \\
                \text{subject to:} \quad & \sum_{b \in \Gamma_{\mathsf{B}}} =
                                           \mathbb{I}, \qquad \qquad
                                           \forall y \in \Sigma_{\mathsf{B}}, \\
                                   \quad & B_b^y \in \text{Pos}(\mathcal{B}),
                                   \qquad \forall y \in \Sigma_{\mathsf{B}}, \
                                   \forall b \in \Gamma_{\mathsf{B}}.
\end{align*}
$$




In [7]:
import cvxpy
import numpy as np
import os
import sys
module_path = os.path.abspath(os.path.join("..", "..", "..", ".."))
if module_path not in sys.path:
    sys.path.append(module_path)
from toqito.nonlocal_games.two_player_quantum_lower_bound import two_player_quantum_lower_bound

In [1]:
pip install toqito

     |████████████████████████████████| 133kB 1.4MB/s eta 0:00:01
     |████████████████████████████████| 30.3MB 12.6MB/s eta 0:00:01     |█████████████████████████▋      | 24.2MB 12.6MB/s eta 0:00:01
     |████████████████████████████████| 245kB 12.0MB/s eta 0:00:01
     |████████████████████████████████| 3.3MB 7.7MB/s eta 0:00:01
     |████████████████████████████████| 4.3MB 20.8MB/s eta 0:00:01
     |████████████████████████████████| 1.6MB 15.9MB/s eta 0:00:01
     |████████████████████████████████| 3.1MB 7.0MB/s eta 0:00:01
  Created wheel for cvx: filename=cvx-0.1.3-cp37-none-any.whl size=3103 sha256=403317b6fc53f4b6392459e4fb95fa9cfb906a4c73cd8be0bfc32badaca5d3aa
  Stored in directory: /Users/vincent.russo/Library/Caches/pip/wheels/a6/4e/e3/a3990e5666a46ec60a27e9521b72631be12838fd6550864d05
  Created wheel for picos: filename=PICOS-2.0.0-cp37-none-any.whl size=321001 sha256=88ba41c22e7be0c58fb01e5f594f19054b6b6a4279a4eac322eb19b16ccc30fa
  Stored in directory: /Users/vincent.russ

In [5]:
from toqito.nonlocal_games.two_player_quantum_lower_bound import two_player_quantum_lower_bound
import numpy as np
dim = 2
num_alice_inputs, num_alice_outputs = 2, 2
num_bob_inputs, num_bob_outputs = 2, 2
prob_mat = np.array([[1 / 4, 1 / 4], [1 / 4, 1 / 4]])
pred_mat = np.zeros(
    (num_alice_outputs, num_bob_outputs, num_alice_inputs, num_bob_inputs)
)

for a_alice in range(num_alice_outputs):
    for b_bob in range(num_bob_outputs):
        for x_alice in range(num_alice_inputs):
            for y_bob in range(num_bob_inputs):
                if np.mod(a_alice + b_bob + x_alice * y_bob, dim) == 0:
                    pred_mat[a_alice, b_bob, x_alice, y_bob] = 1

lower_bound = two_player_quantum_lower_bound(dim, prob_mat, pred_mat)
print(lower_bound)
#self.assertEqual(
#    np.isclose(lower_bound, np.cos(np.pi / 8) ** 2, rtol=1e-02), True
#)

0.8535533840915608


In [11]:
cvxpy.__version__

'1.0.28'

In [20]:
pip install -e git://github.com/cvxgrp/cvxpy.git@master#egg=cvxpy


Obtaining cvxpy from git+git://github.com/cvxgrp/cvxpy.git@master#egg=cvxpy
  Cloning git://github.com/cvxgrp/cvxpy.git (to revision master) to ./src/cvxpy
  Running command git clone -q git://github.com/cvxgrp/cvxpy.git /Users/vincent.russo/Desktop/toqito/docs/notebooks/nonlocal_games/nonlocal_games/src/cvxpy
  Installing build dependencies ... error
  ERROR: Command errored out with exit status 1:
   command: /Library/Frameworks/Python.framework/Versions/3.7/bin/python3.7 /Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pip install --ignore-installed --no-user --prefix /private/var/folders/pn/8cj1p7rj2yd2fq36_5mjs67rtn1k46/T/pip-build-env-p1idvhtd/overlay --no-warn-script-location --no-binary :none: --only-binary :none: -i https://pypi.org/simple -- 'numpy >= 1.15' 'scipy >= 1.1.0' 'setuptools>=40.8.0' wheel
       cwd: None
  Complete output (51 lines):
  Traceback (most recent call last):
    File "/Library/Frameworks/Python.framework/Versions/3.7/lib/p